# Global pairwise alignments

Use following matrix:

* `S(0,j) = j*g`
* `S(i,0) = i*g`
* `S(i,j) = max(S(i-1,j-1) + d(a(i),b(j)), S(i-1,j) + g, S(i,j-1) + g)`

Use PAM250 as score matrix. Implement tracing matrix for path taken by max operator in each cell. Can store this using relative coordinates in each step: (-1,-1), (-1,0) or (0,-1).

Use gap penalty of minus 5.

In [1]:
import numpy as np

In [2]:
PAM250 = {
'A': {'A': 2,  'C': -2, 'D':  0, 'E': 0, 'F': -3, 'G':  1, 'H': -1, 'I': -1, 'K': -1, 'L': -2, 'M': -1, 'N':  0, 'P':  1, 'Q':  0, 'R': -2, 'S':  1, 'T':  1, 'V':  0, 'W': -6, 'Y': -3},
'C': {'A': -2, 'C': 12, 'D': -5, 'E':-5, 'F': -4, 'G': -3, 'H': -3, 'I': -2, 'K': -5, 'L': -6, 'M': -5, 'N': -4, 'P': -3, 'Q': -5, 'R': -4, 'S':  0, 'T': -2, 'V': -2, 'W': -8, 'Y':  0},
'D': {'A': 0,  'C': -5, 'D':  4, 'E': 3, 'F': -6, 'G':  1, 'H':  1, 'I': -2, 'K':  0, 'L': -4, 'M': -3, 'N':  2, 'P': -1, 'Q':  2, 'R': -1, 'S':  0, 'T':  0, 'V': -2, 'W': -7, 'Y': -4},
'E': {'A': 0,  'C': -5, 'D':  3, 'E': 4, 'F': -5, 'G':  0, 'H':  1, 'I': -2, 'K':  0, 'L': -3, 'M': -2, 'N':  1, 'P': -1, 'Q':  2, 'R': -1, 'S':  0, 'T':  0, 'V': -2, 'W': -7, 'Y': -4},
'F': {'A': -3, 'C': -4, 'D': -6, 'E':-5, 'F':  9, 'G': -5, 'H': -2, 'I':  1, 'K': -5, 'L':  2, 'M':  0, 'N': -3, 'P': -5, 'Q': -5, 'R': -4, 'S': -3, 'T': -3, 'V': -1, 'W':  0, 'Y':  7},
'G': {'A': 1,  'C': -3, 'D':  1, 'E': 0, 'F': -5, 'G':  5, 'H': -2, 'I': -3, 'K': -2, 'L': -4, 'M': -3, 'N':  0, 'P':  0, 'Q': -1, 'R': -3, 'S':  1, 'T':  0, 'V': -1, 'W': -7, 'Y': -5},
'H': {'A': -1, 'C': -3, 'D':  1, 'E': 1, 'F': -2, 'G': -2, 'H':  6, 'I': -2, 'K':  0, 'L': -2, 'M': -2, 'N':  2, 'P':  0, 'Q':  3, 'R':  2, 'S': -1, 'T': -1, 'V': -2, 'W': -3, 'Y':  0},
'I': {'A': -1, 'C': -2, 'D': -2, 'E':-2, 'F':  1, 'G': -3, 'H': -2, 'I':  5, 'K': -2, 'L':  2, 'M':  2, 'N': -2, 'P': -2, 'Q': -2, 'R': -2, 'S': -1, 'T':  0, 'V':  4, 'W': -5, 'Y': -1},
'K': {'A': -1, 'C': -5, 'D':  0, 'E': 0, 'F': -5, 'G': -2, 'H':  0, 'I': -2, 'K':  5, 'L': -3, 'M':  0, 'N':  1, 'P': -1, 'Q':  1, 'R':  3, 'S':  0, 'T':  0, 'V': -2, 'W': -3, 'Y': -4},
'L': {'A': -2, 'C': -6, 'D': -4, 'E':-3, 'F':  2, 'G': -4, 'H': -2, 'I':  2, 'K': -3, 'L':  6, 'M':  4, 'N': -3, 'P': -3, 'Q': -2, 'R': -3, 'S': -3, 'T': -2, 'V':  2, 'W': -2, 'Y': -1},
'M': {'A': -1, 'C': -5, 'D': -3, 'E':-2, 'F':  0, 'G': -3, 'H': -2, 'I':  2, 'K':  0, 'L':  4, 'M':  6, 'N': -2, 'P': -2, 'Q': -1, 'R':  0, 'S': -2, 'T': -1, 'V':  2, 'W': -4, 'Y': -2},
'N': {'A': 0,  'C': -4, 'D':  2, 'E': 1, 'F': -3, 'G':  0, 'H':  2, 'I': -2, 'K':  1, 'L': -3, 'M': -2, 'N':  2, 'P':  0, 'Q':  1, 'R':  0, 'S':  1, 'T':  0, 'V': -2, 'W': -4, 'Y': -2},
'P': {'A': 1,  'C': -3, 'D': -1, 'E':-1, 'F': -5, 'G':  0, 'H':  0, 'I': -2, 'K': -1, 'L': -3, 'M': -2, 'N':  0, 'P':  6, 'Q':  0, 'R':  0, 'S':  1, 'T':  0, 'V': -1, 'W': -6, 'Y': -5},
'Q': {'A': 0,  'C': -5, 'D':  2, 'E': 2, 'F': -5, 'G': -1, 'H':  3, 'I': -2, 'K':  1, 'L': -2, 'M': -1, 'N':  1, 'P':  0, 'Q':  4, 'R':  1, 'S': -1, 'T': -1, 'V': -2, 'W': -5, 'Y': -4},
'R': {'A': -2, 'C': -4, 'D': -1, 'E':-1, 'F': -4, 'G': -3, 'H':  2, 'I': -2, 'K':  3, 'L': -3, 'M':  0, 'N':  0, 'P':  0, 'Q':  1, 'R':  6, 'S':  0, 'T': -1, 'V': -2, 'W':  2, 'Y': -4},
'S': {'A': 1,  'C':  0, 'D':  0, 'E': 0, 'F': -3, 'G':  1, 'H': -1, 'I': -1, 'K':  0, 'L': -3, 'M': -2, 'N':  1, 'P':  1, 'Q': -1, 'R':  0, 'S':  2, 'T':  1, 'V': -1, 'W': -2, 'Y': -3},
'T': {'A': 1,  'C': -2, 'D':  0, 'E': 0, 'F': -3, 'G':  0, 'H': -1, 'I':  0, 'K':  0, 'L': -2, 'M': -1, 'N':  0, 'P':  0, 'Q': -1, 'R': -1, 'S':  1, 'T':  3, 'V':  0, 'W': -5, 'Y': -3},
'V': {'A': 0,  'C': -2, 'D': -2, 'E':-2, 'F': -1, 'G': -1, 'H': -2, 'I':  4, 'K': -2, 'L':  2, 'M':  2, 'N': -2, 'P': -1, 'Q': -2, 'R': -2, 'S': -1, 'T':  0, 'V':  4, 'W': -6, 'Y': -2},
'W': {'A': -6, 'C': -8, 'D': -7, 'E':-7, 'F':  0, 'G': -7, 'H': -3, 'I': -5, 'K': -3, 'L': -2, 'M': -4, 'N': -4, 'P': -6, 'Q': -5, 'R':  2, 'S': -2, 'T': -5, 'V': -6, 'W': 17, 'Y':  0},
'Y': {'A': -3, 'C':  0, 'D': -4, 'E':-4, 'F':  7, 'G': -5, 'H':  0, 'I': -1, 'K': -4, 'L': -1, 'M': -2, 'N': -2, 'P': -5, 'Q': -4, 'R': -4, 'S': -3, 'T': -3, 'V': -2, 'W':  0, 'Y': 10}}

In [52]:
seq1 = "ACG"
seq2 = "GAC"
gap_score = -1

## Setup

In [63]:
def setup_matrices(seq1, seq2):

    score_m = np.zeros((len(seq1)+1, len(seq2)+1))
    dir_m = np.chararray((len(seq1)+1, len(seq2)+1), itemsize=5)
    

    for i in range(1,len(seq1)+1):
        score_m[i,0] = gap_score * i
        dir_m[i,0] = '-1,0'

    for j in range(1,len(seq2)+1):
        score_m[0,j] = gap_score * j
        dir_m[0,j] = '0,-1'

    dir_m[0,0] = '0,0'
        
    return (score_m, dir_m)


In [66]:

def calculate_matrices(seq1, seq2, scoring):
    score_m, dir_m = setup_matrices(seq1, seq2)

    for i in range(1, len(seq1)+1):
        for j in range(1, len(seq2)+1):
            aa1 = seq1[i-1]
            aa2 = seq2[j-1]
            comp_score = scoring[aa1][aa2]

            print("Coords, i: {} j: {}".format(i,j))
            top_score = score_m[i-1, j] + gap_score
            left_score = score_m[i, j-1] + gap_score
            match_score = score_m[i-1, j-1] + comp_score
            scores = [top_score, left_score, match_score]

            print(scores)
            
            if top_score == max(scores):
                score_m[i,j] = top_score
                dir_m[i,j] = '-1,0'
            elif left_score == max(scores):
                score_m[i,j] = left_score
                dir_m[i,j] = '0,-1'
            elif match_score == max(scores):
                score_m[i,j] = match_score
                dir_m[i,j] = '-1,-1'
            else:
                raise Error("Score matching failed for scores: {}".format(scores))
    return dir_m, score_m


In [72]:
get_alignment('WA', 'WAW', PAM250)

Coords, i: 1 j: 1
[-2.0, -2.0, 17.0]
Coords, i: 1 j: 2
[-3.0, 16.0, -7.0]
Coords, i: 1 j: 3
[-4.0, 15.0, 15.0]


KeyError: 'B'

In [ ]:
score_m

# Calculate alignment from path

In [58]:
def alignment_from_path(seq1, seq2, dir_m):
    
    path1 = list()
    path2 = list()
    
    i = len(seq1)
    j = len(seq2)
    while i > 0 or j > 0:
        curr_val = [int(v) for v in dir_m[i,j].decode('utf-8').split(',')]
  
        

        if curr_val[0] == -1 and curr_val[1] == -1:
            print('diag')
            i -= 1
            j -= 1
            path1 += seq1[i]
            path2 += seq2[j]
        elif curr_val[0] == -1:
            
            print('up')
            i -= 1            
            path1 += seq1[i]
            path2 += '-'
        else:
            print('left')
            j -= 1
            path1 += '-'
            path2 += seq2[j]
            
    return ''.join(reversed(path1)), ''.join(reversed(path2))
            

In [45]:
def get_alignment(seq1, seq2, scoring):
    dir_m, score_m = calculate_matrices(seq1, seq2, scoring)
    print(dir_m)
    print(score_m)
    score = score_m[len(seq1), len(seq2)]
    align1, align2 = alignment_from_path(seq1, seq2, dir_m)
    return (align1, align2, score)

In [48]:
get_alignment('A', 'GA', PAM250)

[-10.0, -10.0, 1.0]
[-5.0, -4.0, -4.0]
[[b'0,0' b'0,-1' '']
 [b'-1,0' b'-1,-1' b'0,-1']]
[[ 0. -5.  0.]
 [-5.  1. -4.]]
left
diag


('A-', 'GA', -4.0)